# Lab 9: 비정형 데이터와 Cortex LLM Function

👉 이번 수업에서는 제공된 PDF 문서를 검토하고, Snowflake Cortex Function을 사용하여 이 문서를 파싱하고 정보를 테이블에 작성합니다. 두 번째 Cortex Function은 테이블의 데이터 행에 라벨을 붙이거나 분류하는 데 사용됩니다. 분석 쿼리를 작성하고 이를 Streamlit을 사용하여 시각화하여 카테고리별 정보 분포를 보여줍니다. 마지막으로 다른 세 가지 Cortex Function의 기능을 간략히 살펴볼 것입니다.

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
session.use_database(f'{user}_GARDEN_PLANTS')
session.use_schema('VEGGIES')
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

## Snowflake에서 비정형 데이터 작업 📓

이 과정의 이전 섹션에서는 정형 데이터(컬럼과 행을 생각하면 됨)와 반정형 데이터(JSON 형식을 생각하면 됨)를 살펴보았습니다. Snowflake는 [**비정형 데이터**](https://docs.snowflake.com/ko/user-guide/unstructured-intro)를 다루기 위한 기능과 함수도 포함하고 있습니다. 분석가들은 2025년까지 전 세계 데이터의 80%가 비정형 데이터로 구성될 것이라 예상합니다.

그렇다면 비정형 데이터란 무엇일까요?

**비정형 데이터**는 사전 정의된 데이터 모델이나 스키마에 맞지 않는 정보를 의미합니다. 일반적으로 양식 응답, 소셜 미디어 대화와 같은 텍스트가 큰 데이터뿐만 아니라 이미지, 비디오, 오디오를 포함합니다. VCF(유전체학), KDF(반도체), HDF5(항공우주)와 같은 산업별 파일 유형도 이 범주에 포함됩니다.

Snowflake AI Data Cloud는 **비정형 데이터** 파일에 접근하고, 이를 공유하며, 처리할 수 있도록 지원합니다. 

## 비정형 데이터 시나리오 📓

잠시만 생각해 보세요. 시간이 지나면서 가꾸는 식물에 대한 흥미로운 사실과 세부 정보를 수십 가지 수집하고, 이 "조각(snippets)" 정보를 PDF 파일로 노트북(labtop)에 저장했다고 가정해 봅시다. 이러한 데이터를 Snowflake로 가져와 이미 구축한 정보와 함께 사용할 수 있다면 정말 좋지 않을까요?

이 실습에서 다룰 시나리오가가 바로 이것입니다. 다음 작업을 수행하게 됩니다:
1. Cortex LLM Function을 사용하여 스테이지된 PDF파일에서 정보를 파싱합니다.
1. 데이터를 추출하고 Snowflake의 새 테이블에 넣습니다.
1. 이 데이터를 이용하여 분석하고 생성 작업을 하는 Snowflake Cortex LLM Function을 알아봅니다. :robot_face:

### 제공된 PDF 파일의 목록 생성 🥋

약 130개의 PDF 파일 모음이 식물에 대한 흥미로운 사실과 세부 정보를 담고 있으며, Education Services 팀이 이를 **common_db.resources**의 **course_files** 스테이지에 업로드했습니다.

아래 SQL 코드를 **수정**하고 **셀을 실행**하여 지정된 스테이지에 제공된 PDF 파일을 `LIST`하세요:
- 해시 문자 `(#)`를 적절한 명령과 구문으로 교체하여 올바른 SQL 구문을 작성합니다.
- SQL 셀을 실행합니다.

In [ ]:
list @common_db.resources.course_files/garden_kb

## Directory Table 📓

비정형 데이터를 다룰 때 **Directory Table**은 매우 유용하며, `LIST` 명령을 사용하는 것보다 여러 면에서 더 우수합니다.

[Directory Table](https://docs.snowflake.com/ko/user-guide/data-load-dirtables)은 스테이지 위에 계층적으로 존재하는 암시적 오브젝트로, 별도의 데이터베이스 오브젝트는 아닙니다. 개념적으로 External Table과 유사하며, 스테이지 내 데이터 파일에 대한 파일 수준 메타데이터를 저장합니다. Internal Stage와 External Stage 모두에 지원됩니다. Directory Table의 가장 큰 장점은 디렉토리의 내용을 테이블처럼 쿼리할 수 있다는 점입니다! Directory Table은 스테이지를 생성할 때 옵션을 활성화하거나, 생성 후 스테이지를 변경하는 것으로 간단히 생성할 수 있습니다.

- 사실, Lab 8에서 이미 Directory Table을 생성했을 것입니다. 이는 Snowsight 마법사를 사용하여 스테이지 오브젝트를 생성할 때 기본적으로 포함된 옵션이기 때문입니다.

Directory Table의 출력을 테이블 함수로 액세스할 수 있으며, 파일 이름과 위치를 Snowflake의 다양한 함수로 쉽게 처리할 수 있습니다.


### Directory Table 쿼리하기 🥋

다음 쿼리의 구조를 살펴봅시다:
- 일반적인 `SELECT` 구문입니다.
- 테이블 함수에 접근하기 위해 **DIRECTORY** 키워드를 사용하는 점에 주목하세요.
- 이 경우 반환할 컬럼을 `*` ("star 모두")로 선택할 수 있습니다.
- 결과를 필터링하여 원하는 데이터를 선택할 수도 있습니다. 예: 이 스테이지의 "garden_kb" 하위 디렉토리 아래에 있는 파일반 반환

이 쿼리를 실행하고 출력 결과를 검토하며, 이 스테이지 위치에 포함된 파일에 대해 반환되는 정보의 유형을 확인하세요.

In [ ]:
SELECT *    
FROM DIRECTORY('@common_db.resources.course_files')
WHERE CONTAINS(relative_path, 'garden_kb/');

## 제공된 PDF 파일 검토. 🥋

### 어떤 정원 관리(gardening) 인사이트가 수집되었나요?

앞서 언급했듯이, Education Services 팀이 약 130개의 PDF 파일 모음을 업로드했습니다. 이 파일들은 식물에 대한 흥미로운 사실과 세부 정보를 담고 있습니다.

- PDF 파일 모음의 이름은 **snippet_1.pdf**에서 **snippet_129.pdf**까지입니다.

이런 "사실 정보(factoid)" 파일들이 어떤 정보를 포함하고 있는지 궁금할 수도 있습니다!

다음은 이 PDF 파일 모음에서 제공된 식물 및 정원 관리 지식의 예입니다. 이 텍스트는 모음의 첫 번째 파일에서 가져왔으며 **artichokes**와 관련이 있습니다.

**artichokes**를 키우고 있나요? 여기 몇 가지 좋은 팁이 있습니다!

![snippet sample (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_artichokes_1.png)

### 식물 관련 "사실 정보" PDF 파일 중 하나 다운로드 및 검토 📓

위에서 식물 및 정원 관리 지식 모음에 포함된 PDF 파일의 콘텐츠 예제를 보았습니다. 이번에는 다른 PDF 파일을 다운로드하여 열어보고 검토해 보겠습니다.

- 이번 예제에서는 **snippet_9.pdf** 파일을 살펴보겠습니다.

- 이 예제의 SQL 코드(Python에 내장된)는 Snowflake 기능을 활용하여 스테이지에 저장된 파일에 대한 링크를 생성합니다. 이 링크는 Snowflake "외부"에서도 접근할 수 있습니다. 이는 **비정형** 데이터를 처리할 때 유용합니다. [GET_PRESIGNED_URL](https://docs.snowflake.com/ko/sql-reference/functions/get_presigned_url) 함수에 대해 더 자세히 알아보시려면 Snowflake 설명서를 참조하세요.

### 다음 Python 코드 셀을 실행하고 생성된 링크를 사용해 파일을 다운로드합니다. 🥋

생성된 링크를 **오른쪽 클릭**하여 새 탭 또는 브라우저 창에서 엽니다.(링크를 클락만 하면 작동하지 않을 수 있습니다)

PDF 파일이 브라우저에서 자동으로 열리지 않는 경우, 로컬 컴퓨터의 PDF 뷰어 애플리케이션에서 파일을 **엽니다**

- 이번 PDF 파일에서는 여러분이 좋아하는 채소 - 아스파라거스에 대한 **알짜 정보**를 확인해 보겠습니다! :leafy_green:

- 첫 번째 예제를 살펴본 후, 번호가 1부터 129까지 있는 다른 파일들도 자유롭게 확인해 보세요. 

In [ ]:
snowpark_df = session.sql("SELECT GET_PRESIGNED_URL(@common_db.resources.course_files, 'garden_kb/snippet_9.pdf')")
collected_data = snowpark_df.collect()
st.write('Open the following link in a new browser tab or window and review.')
st.write(collected_data[0][0])
st.write('Asparagus. Who knew!!!')

## Snowflake Cortex AI 소개 📓

Snowflake AI Data Cloud는 Mistral, Reka, Meta, Google과 같은 기업의 연구진이 학습시킨 업계 선도적인 대형 언어 모델(LLM)에 즉시 접근할 수 있는 다양한 기눙과 함수를 제공합니다. 여기에는 Snowflake가 개발한 개방형 엔터프라이즈급 모델인 Snowflake Arctic도 포함됩니다.

이 LLM들은 Snowflake가 완전히 호스팅 및 관리하므로, 별도의 **설정이 필요 없습니다**. 데이터는 Snowflake 내에 안전하게 유지되며, 기대하는 성능, 확장성, 데이터 거버넌스를 제공합니다. 

💡 아래 기능의 [출시 상태 및 이용 가능 여부](https://docs.snowflake.com/en/guides-overview-ai-features)에 대한 자세한 내용은 Snowflake 설명서를 참조하세요.

![Snowflake generative AI (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_gen_ai_1.png)

### Snowflake Cortex LLM function.

Snowflake AI 옵션 중 하나는 **Cortex LLM Function**으로, SQL 함수 형태로 제공되며 Python에서도 사용할 수 있어 강력한 기능에 쉽게 접근할 수 있습니다!

Cortex LLM Function은 다음과 같은 범주로 나눌 수 있습니다:

- 작업별 특화 함수

- 헬퍼 함수

- `COMPLETE` 함수

👉 이번 실습의 나머지 과정에서는 이러한 함수 중 일부를 활용하여 식물 및 정원 관리 데이터를 "처리"하는 방법을 살펴보겠습니다.

## `PARSE_DOCUMENT()`를 활용한 텍스트 추출 📓

Cortex의 [parse_document()](https://docs.snowflake.com/user-guide/snowflake-cortex/parse-document)는 Cortex LLM의 작업별 특화 함수로, Internal Stage 또는 External Stage에 저장된 문서에서 텍스트나 레이아웃을 추출할 수 있는 기능을 제공합니다.  

이 함수는 SQL 함수로, Snowflake에서 완전히 호스팅 및 관리되므로 별도의 설정이 필요하지 않습니다. 즉, PDF 문서가 저장된 스테이지를 `PARSE_DOCUMENT` 함수에 지정하기만 하면 텍스트 또는 레이아웃 데이터를 추출할 수 있습니다. 간단히 말하면, 이 함수에 필요한 것은 다음과 같습니다:

- 읽어올 스테이지의 이름

- 해당 스테이지에서 텍스트를 추출하려는 PDF 문서(현재는 PDF 파일만 지원)

- 문서 레이아웃을 읽을 수도 있지만, 우리는 텍스트 추출을 다룰 수 있는 **OCR** 모드를 선택할 것입니다.

💡 **팁**: 더욱 정교한 문서 추출 사례를 위해 이 함수의 **LAYOUT** 모드를 확인하거나 Snowflake의 [Document AI](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/overview) 서비스를 검토해 보세요.

### 텍스트 추출 예제 🥋

다음 SQL 구문을 검토하세요:

- `PARSE_DOCUMENT()` 함수에 대한 전체 경로 참조를 사용합니다.

- PDF 문서가 저장된 스테이지와 해당 스테이지 내의 특정 파일에 대한 참조를 제공합니다. (이는 이전에 열었던 "아스파라거스" 예제입니다)

이처럼 간단합니다. "빌트인" SQL 함수를 호출하기만 하면, Snowflake가 파일 내용을 열고 읽은 뒤 결과를 반환합니다.

아래 코드를 실행해 보세요:

In [ ]:
SELECT SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
        @common_db.resources.course_files, --읽어올 스테이지의름
        'garden_kb/snippet_9.pdf', --추출할 문서
        {'mode': 'OCR'} --모드: 텍스트 추출 (다른 예: 레이아웃기)
    ) AS output;

### `PARSE_DOCUMENT()` 출력 검토 🥋

이 함수의 출력은 반정형 데이터 형식으로 제공된다는 점에 주목하세요:

![Parse_document() output (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_parse_document_1.png)

출력에는 **content** 와 **metadata** 필드가 포함되어 있습니다. 

💡 **팁**: **Lab 8**에서 배운 것처럼, `:` 연산자를 사용해 이러한 반정형 데이터 구조 안에 포함된 중첩 필드에 접근할 수 있다는 점을 기억하세요!

## 텍스트 추출 워크플로우 구축 📓

식물 및 정원 관리 PDF 문서에서 텍스트를 추출할 수 있는 방법을 알게 되었으니, 이를 Snowflake로 가져와 활용해 보겠습니다.

👉 우리는 수집한 모든 식물 및 정원 관리 정보를 바탕으로 **지식 베이스**를 구축하려고 하며, 이 데이터를 다양한 용도로 활용할 가능성을 모색하고 있습니다.

다음은 워크플로우입니다:

- 지식 베이스 정보는 PDF 파일의 **비정형** 데이터로 시작합니다.

- Cortex의 `PARSE_DOCUMENT()` 함수를 실행하여 PDF 파일에서 텍스트를 추출하고, 이를 **반정형** 데이터 형식으로 변환합니다.

- Snowflake의 구문을 사용해 Cortex 함수의 출력을 파싱하여 **정형** 데이터 콘텐츠로 반환합니다.

### 지식 베이스 데이터를 저장할 테이블 생성 🥋 

먼저, PDF 문서에서 추출된 참조 정보를 저장할 테이블을 생성해야 합니다.

아래 코드를 실행하여 **(animal)_GARDEN_PLANTS.VEGGIES** 스키마에 테이블을 생성합니다. 참고로 PDF 파일의 내용이 언급하는 식물의 이름을 저장할 컬럼도 포함될 것입니다.

In [ ]:
CREATE OR REPLACE TABLE vegetable_knowledge_base (
    source_document STRING, -- the document name
    insight STRING,         -- the "factoid" contained with the file
    plant_name STRING       -- the name of the plant the "factoid" references
);

### 데이터 추출 및 새로운 테이블에 `INSERT` 🥋

이제 이번 실습에서 배운 몇가지 기능과 함수를 함께 살펴보겠습니다.

다음 `INSERT` 구문은 **Directory Table**과 `PARSE_DOCUMENT()` 함수를 활용합니다:

- **Directory Table**은 **common_db.resources.course_files** 스테이지의 **garden_kb** 하위 디렉토리에 있는 각 PDF 파일의 이름을 반환합니다.

- 각 PDF 문서의 경로가 `PARSE_DOCUMENT()` 함수에 전달되어 텍스트를 추출합니다.

- 추출된 텍스트는 반정형 형식으로 반환되며, 그 중 **content** 엘레멘트를 추출하여 다음과 같이 `STRING`으로 변환합니다: `:content::STRING as extract`

아래 코드를 **실행**하여 각 PDF 파일의 이름과 추출된 텍스트를 새 테이블에 작성합니다.

In [ ]:
INSERT INTO vegetable_knowledge_base (source_document, insight)
    SELECT 
        split_part(relative_path,'/',-1) as file_name, 
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            @common_db.resources.course_files,
            relative_path,
            {'mode': 'OCR'}
        ):content::STRING as extract    
    from directory('@common_db.resources.course_files')
    where contains(relative_path, 'garden_kb/')
;

### 작업 확인  🎯

새 테이블에 **129**개의 행이 삽입되었는지 확인하세요. 이제 테이블에 저장된 데이터의 "형태"를 살펴봅시다.

- 아래 쿼리 조각을 **수정하여** 새로 만든 지식 베이스 테이블에서 **모든** 행을 반환하도록 하세요.

In [ ]:
SELECT *
FROM vegetable_knowledge_base

### 문제가 발생했습니다!

PDF 파일에서 데이터를 추출하여 테이블에 기록하는 데는 성공했지만, 다음과 같은 문제가 있습니다...

**INSIGHT** 컬럼의 내용을 검토하지 않으면 각 행의 정보가 어떤 식물과 관련이 있는지 알 수 없습니다. **PLANT_NAME** 컬럼이 모두 비어 있기 때문입니다. 이는 지식 베이스를 구현할 때 큰 문제입니다!

물론, 각 행을 읽고 **PLANT_NAME** 컬럼을 수동으로 업데이트할 수도 있지만, 이 테이블에 129개의 행이 있기 때문에 시간이 많이 걸릴 뿐만 아니라, 더 큰 데이터 세트에서는 실행 불가능한 방법입니다.

![Knowledge base table 1 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_kb_table_query_1.png)

## 추출한 텍스트에 `CLASSIFY_TEXT()` 활용 📓

다행히도, Snowflake Cortex에는 [CLASSIFY_TEXT()](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex)라는 LLM 함수가 포함되어 있습니다. 이름에서 알 수 있듯, 이 함수는 제공된 자유 형식 텍스트 데이터를 분류합니다.

- SQL에서 간단히 호출할 수 있으며, 다른 Cortex LLM Function과 마찬가지로 별도의 설정이 필요 없습니다. Snowflake에서 기본 제공됩니다.

- 이 함수는 JSON 오브젝트가 포함된 문자열을 반환합니다. JSON 오브젝트는 입력 텍스트가 분류된 카테고리를 포함합니다. 만약 잘못된 인수가 제공되면 오류를 반환합니다.

![Classify text usage (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_classify_text_1.png)

### 분류 예제 🥋

사용 사례와 관련된 다음 예제를 검토합니다. 입력 문자열로 **apple**을 Cortex LLM function에 전달하여 검토하고, 이를 정확히 **fruit**, **veggie**, 또는 **flower**로 분류하기를 기대합니다.

아래 코드를 실행해 보세요.

- 결과가 정확했나요?

- 이번에는 **tomato**를 입력값으로 사용해 보세요 :grinning:

In [ ]:
SELECT SNOWFLAKE.CORTEX.CLASSIFY_TEXT('apple', ['fruit', 'veggies', 'flowers']);

### 확장된 분류 사용 사례 📓

훌륭합니다! 이제 테이블의 129개 행을 수동으로 업데이트할 필요가 없고, Snowflake가 무거운 작업을 대신 처리할 수 있습니다.

- `UPDATE` 구문을 작성해야 합니다.

- 테이블의 각 행에 대해 **INSIGHT** 컬럼을 입력으로 전달합니다.

- 이전 실습에서 생성 및 로드된 **VEGETABLE_DETAILS** 테이블에서 참조하는 모든 알려진 식물의 이름 목록을 구성할 수 있습니다.

- 반정형 데이터 출력으로 들어가 라벨 값을 추출할 수 있습니다. 

### 분류 실행 🥋

다음 SQL 구문을 검토하세요:

- `CLASSIFY_TEXT()` 함수에 대한 전체 경로 참조를 사용합니다.

- **4번 라인**의 구문이 생소할 수 있지만, 이 부분은 단지 **vegetable_details** 테이블에서 식물 이름의 `ARRAY`를 생성하여 Cortex LLM Function에 전달하는 역할을 합니다. 
    - 이는 `Artichoke...Zucchini`를 일일이 수동으로 입력하는 대신 시간을 절약해 줍니다!

아래 코드를 실행해 보세요:

In [ ]:
UPDATE vegetable_knowledge_base
SET plant_name = SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
    insight, 
    (SELECT ARRAY_AGG(plant_name) WITHIN GROUP (ORDER BY plant_name ASC) FROM vegetable_details) -- ASSEMBLE CATEGORIES
):label::STRING
WHERE insight IS NOT NULL 
AND insight <> '';

### 작업 확인 🥋

**129**개의 행이 지식 베이스 테이블에서 **업데이트**되었는지 확인합니다. 이제 테이블에 있는 데이터를 살펴봅시다.

- 아래 쿼리를 실행하여 지식 베이스 테이블의 모든 행을 반환합니다.

- **PLANT_NAME** 컬럼이 **모든** 행에 대해 채워졌는지 확인합니다.

In [ ]:
SELECT *
FROM vegetable_knowledge_base;

## 지식 베이스 데이터 분석 📓

제공된 PDF 파일에서 로드된 지식 베이스 테이블과 각 행이 식물 이름에 따라 분류되었으니, 이 데이터의 분포를 이해하는 것이 유용할 것입니다.

- 각 식물에 대해 얼마나 많은 "사실 정보"가 있나요?

- 일부 식물에 대한 정보가 다른 식물보다 더 많나요?

- 일부 식물 모음에 대해 누락된 데이터가 있나요?

이 모든 정보는 지식 베이스를 확장하기 위한 우선순위를 결정하는 데 도움을 줄 수 있습니다.

### 분석 쿼리 실행 🥋

다음 쿼리는 Snowflake에서 지원하는 [JOIN](https://docs.snowflake.com/ko/sql-reference/constructs/join) 작업 유형 중 하나의 예입니다.

여기서는 **vegetable_details** 테이블에서 식물 이름 전체 목록을 가져오고, 각 식물에 대해 **vegetable_knowledge_base** 테이블의 행의 수를 계산하려고 합니다. `LEFT OUTER JOIN`을 사용하면 **vegetable_knowledge_base**에 특정 식물에 대한 행이 없더라도, 결과에서 완전히 생략하지 않고 제로 값을 할당할 수 있습니다.

아래 쿼리를 실행하고 검토하여 식물별 기사 수를 확인하세요. 

💡 **팁**:  이 셀의 이름을 **knowledge_base_analytical_query**로 지정하여 나중에 출력 결과를 쉽게 참조할 수 있도록 하겠습니다.

In [ ]:
SELECT a.plant_name, 
       nvl(count(b.*),0) AS kb_article_count
FROM vegetable_details a
LEFT OUTER JOIN vegetable_knowledge_base b
ON a.plant_name = b.plant_name
GROUP BY a.plant_name
ORDER BY 1;

## Streamlit in Snowflake 📓

[Streamlit](https://streamlit.io/)은 머신 러닝과 데이터 과학을 위한 맞춤형 웹 애플리케이션을 쉽게 만들고 공유할 수 있도록 도와주는 오픈 소스 Python 라이브러리입니다.

[Streamlit in Snowflake](https://docs.snowflake.com/ko/developer-guide/streamlit/about-streamlit)은 이 기술을 Snowflake AI Data Cloud에서 구현한 것입니다. 

- Snowflake가 Streamlit 앱을 위한 기본 컴퓨팅 및 스토리지를 관리합니다.

- Streamlit 앱은 Snowflake의 오브젝트이며, 역할 기반 액세스 제어(RBAC)를 통해 Streamlit 앱에 대한 접근을 관리합니다.

- Streamlit 앱은 Snowflake 웨어하우스에서 실행되며, Internal Stage를 사용해 파일과 데이터를 저장합니다.

### Streamlit을 사용한 데이터 시각화 🥋

Streamlit의 강점 중 하나는 데이터를 시각화하고 상호작용하는 것이 매우 쉽다는 점입니다. 이미 이 과정의 각 실습 마지막에 있는 **퀴즈** 섹션에서 Streamlit을 사용해 보셨을 것입니다. 간단한 Python 코드 몇 줄만으로 데이터를 생동감있게 보여주는 깔끔하고 매력적인 차트와 그래프를 만들 수 있습니다. 

아래 Python 셀을 실행하여 방금 실행한 분석 쿼리 결과를 시각화하는 Streamlit 막대 차트를 생성해 보세요. 얼마나 적은 코드로 이 차트를 만드는지 주목해 보세요.

- 어떤 식물이 지식 베이스 기사에서 누락되었나요?

- 어떤 식물이 가장 많은 기사를 갖고 있나요?

💡 **팁**: 이 과정의 앞에서 배운 것처럼, Snowflake Notebook에서는 이전 셀의 결과를 나중에 다른 셀에서 참조할 수 있습니다. 아래 코드는 방금 실행한 분석 SQL 쿼리 셀(**knowledge_base_analytical_query**)의 출력을 참조하여 **pandas** Dataframe을 생성한 후, 이를 Streamlit으로 전달합니다.

In [ ]:
import streamlit as st
import pandas as pd

chart_data = knowledge_base_analytical_query.to_pandas() # UTILIZES OUTPUT FROM AN EARLIER SQL CELL !!!

st.header("Knowledge Base Articles Per Plant")
st.bar_chart(chart_data, x="PLANT_NAME", y="KB_ARTICLE_COUNT", color=['#33C4FF'])

## Snowflake AI 와 LLM Function에는 더 많은 기능이 있습니다 📓

Snowflake의 AI(그리고 ML!) 기능에 대해 알아야 할 것이 많지만, 이번 과정에서 모두 다루기에는 한계가 있습니다.

그럼에도 불구하고, 유용한 작업별 특화 Cortex LLM Function 중 몇 가지를 소개하자면 `TRANSLATE()`와 `SUMMARIZE()`가 있습니다. 간단히 살펴보겠습니다:

- [TRANSLATE()](https://docs.snowflake.com/ko/sql-reference/functions/translate-snowflake-cortex) - 주어진 입력 텍스트를 지원되는 언어 간에 번역합니다.

- [SUMMARIZE()](https://docs.snowflake.com/ko/sql-reference/functions/summarize-snowflake-cortex) - 주어진 영어 텍스트를 요약합니다.

이제 지식 베이스 데이터를 활용한 간단한 예제를 살펴보겠습니다.

### `TRANSLATE()` 예제 실행 🥋 

이전 예제에서 살펴본 것처럼, SQL을 사용해 Cortex LLM Function을 호출하는 것은 매우 간단합니다. `TRANSLATE()`를 호출할 때는 다른 SQL 함수와 마찬가지로 다음 입력을 제공합니다:

- 번역할 텍스트가 포함된 문자열

- 텍스트가 현재 작성된 언어의 언어 코드.(예: 프랑스어, 독일어, 이탈리아어, 일본어, 한국어, 스페인어 등의 옵션이 있습니다)

- 텍스트를 번역할 대상 언어의 언어 코드

아래 예제 코드를 **실행**하여, 지식 베이스에서 이름이 `'C'`로 시작하는 식물의 정보를 **영어**에서 **한국어**로 번역한 후, 번역된 버전을 다시 **영어**로 번역해 보세요.

In [ ]:
SELECT plant_name, 
       insight AS original_english_text,
       SNOWFLAKE.CORTEX.TRANSLATE(original_english_text, 'en', 'ko') AS korean_text,
       SNOWFLAKE.CORTEX.TRANSLATE(korean_text, 'ko', 'en') AS english_text_from_korean,
FROM vegetable_knowledge_base
WHERE LEFT(plant_name,1) = 'C'; -- all the plant names beginning with C

### `SUMMARIZE()` 예제 실행 🥋 

`SUMMARIZE()` 함수는 이름 그대로 주어진 영어 텍스트 입력을 요약합니다. 이 함수는 단 하나의 파라미터를 요구하며, 이는 요약하려는 텍스트 문자열입니다.

아래 예제를 **실행**해 보세요. 이 예제는 우리의 모음에서 **Pumpkin**에 초점을 맞춥니다.

- [LISTAGG](https://docs.snowflake.com/ko/sql-reference/functions/listagg)를 사용하여 해당 식물에 대한 모든 "사실 정보"(인사이트)를 하나의 텍스트로 결합합니다.

- `SUMMARIZE()`를 사용해 이 텍스트를 요약합니다.

- Cortex LLM 헬퍼 함수인 [COUNT_TOKENS()](https://docs.snowflake.com/en/sql-reference/functions/count_tokens-snowflake-cortex)를 활용하여 요약 **전**과 **후**의 텍스트의 상대적 크기를 확인합니다. 물론 직접 결과를 검토할 수도 있습니다.

💡 **팁**: 토큰은 Snowflake Cortex LLM Function이 처리하는 텍스트의 가장 작은 단위로, 대략 4개의 문자와 같습니다. 입력 또는 출력 텍스트가 토큰으로 변환되는 비율은 사용하는 모델에 따라 다를 수 있습니다.

아래 코드를 실행하고 결과를 확인해 보세요:

In [ ]:
SELECT listagg(insight) AS all_insights,
        SNOWFLAKE.CORTEX.count_tokens('summarize', all_insights) AS all_insights_tokens,
        SNOWFLAKE.CORTEX.SUMMARIZE(listagg(insight)) AS summary,
        SNOWFLAKE.CORTEX.count_tokens('summarize', summary) AS summary_tokens
FROM vegetable_knowledge_base
WHERE plant_name = 'Pumpkin';

## 마지막으로, `COMPLETE()` 🥋

Cortex LLM Function 중 가장 정교한 것은 [COMPLETE](https://docs.snowflake.com/ko/sql-reference/functions/complete-snowflake-cortex)입니다. 가장 단순한 형태로, 이 함수는 **프롬프트**(텍스트와 해당 텍스트로 수행하고자 하는 지침)를 입력받아, 지원되는 언어 모델 중 하나를 사용하여 **응답**(completion)을 생성합니다.

다음 예제를 통해 작동 방식을 이해해 보세요:

- 이 예제에서는 **snowflake-arctic** 모델을 사용합니다.

- 모델이 응답을 형성하도록 돕기 위해 지침을 제공합니다.(예: "You are an I.T expert")

- 질문을 제시합니다.(예: "Explain what Snowflake is")

- 모델의 응답은 **내재적** 지식, 즉 학습된 데이터에 기반합니다. 우리가 제공하는 추가 정보는 사용되지 않습니다. 

아래 코드를 실행하고 출력 결과를 검토하세요:

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic', 'You are an I.T expert. Explain what Snowflake is.') AS complete_response;

### 지식 베이스 데이터를 `COMPLETE()`로 활용하기 🥋

마지막 예제에서는, 지식 베이스에 저장된 정원 및 식물 관련 **모든** 정보를 집계한 후, 이를 바탕으로 `COMPLETE()`가 답변을 생성하도록 질문을 제시합니다.

### `COMPLETE()` 쿼리 설명

다음 쿼리는 약간 복잡해 보일 수 있지만, 각 부분을 나누어 설명하겠습니다:

#### 섹션 1

- 첫 번째 섹션에는 실행할 샘플 질문이 포함되어 있습니다.

- SQL 셀을 실행할 때마다 하나씩 주석을 해제하여 새로운 질문을 실행해 보세요.

- 이러한 쿼리는 실행 시간이 조금 걸릴 수 있습니다. - 기다려 주세요.

![Complete query section 1 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_complete_query_1.png)

#### 섹션 2

- 두 번째 섹션에서는 프롬프트를 정의합니다. - `COMPLETE()` 함수를 통해 모델에게 전달할 지침과 질문

- 프롬프트에는 텍스트의 서로 다른 블록을 명확히 구분하기 위해 태그를 포함시켜, 모델이 이를 "consumption"하도록 합니다.

![Complete query section 2 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_complete_query_2.png)

#### 섹션 3

- 세 번째 섹션에서는 실제 Cortex 함수를 호출하는 부분입니다.

- 셀의 앞부분에서 정의된 변수를 사용하므로, 이를 통해 더 유연하게 반복 실행할 수 있습니다.

![Complete query section 3 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_complete_query_3.png)

In [ ]:
-- example questions 
SET my_question  = 'which plant would take the shortest amount of time to cook'; -- should take around 30 seconds to complete                                    
--SET my_question  = 'which plants are best for beginner gardeners'; -- should take around 60 seconds to complete 


-- the prompt
SET prompt = 'You are a helpful gardening expert. Use only the supplied information <information> to answer the question posed <question>' || 
              $my_question || '</question> ' ||                 
             ' If you have no supplied information do not answer'; 
                
-- the Cortex function call
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'mixtral-8x7b', --32K context window
    $my_question || 
    '<information>' ||
    (SELECT LISTAGG(insight, ' ') FROM vegetable_knowledge_base) ||
    '</information>'
) AS cortex_output;

## 마지막 QUIZ :mag_right:

## 지식 테스트. :memo: :mag_right:

- 다음 Python 셀을 실행하여 Streamlit 기반 위젯을 표시하고 제시된 질문에 답변하세요.

- 이 질문들은 이 과정에서 다룬 모든 내용에 관한 것일 수 있습니다.

In [ ]:
st.divider()
question = "이전에 작동했던 SELECT 구문을 실행했는데 이제 테이블이 존재하지 않는다는 오류 메시지가 표시된다면 무엇을 확인해야 합니까?"
options = ["아래 보기 중 고르세요...",
           "A) 클라우드 제공업체", 
           "B) 현재 리전", 
           "C) 컨텍스트 역할",
           "D) 컨텍스트 웨어하우스",
           "E) 왼쪽 하단에 표시된 이름 아래의 역할"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'ef3fa590e5b728d1bdaf658d3a18945c'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "다음 오브젝트 중 가장 많은 컴퓨팅 파워를 가진 것은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) Snowflake 데이터베이스", 
           "B) Snowflake 스키마", 
           "C) Snowflake 시퀀스",
           "D) Snowflake 웨어하우스",
           "E) Snowflake 데이터 마트"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'fe742826f7ff02a3e33b22e228ea368e'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "다음 오브젝트 중 저장 컨테이너 계증 구조에서 가장 낮은 단계에 있는 것은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) Snowflake 데이터베이스", 
           "B) Snowflake 스키마", 
           "C) Snowflake 리전",
           "D) Snowflake 계정",
           "E) Snowflake 테이블"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '27e8c3433312fce20b97b074e89590bf'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "아래 설명 중 Snowflake의 External Stage를 설명하는 것은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 스테이지는 테이블의 각 새 행에 대해 고유 ID를 생성하는 카운터로 사용될 수 있습니다. 시작 값과 증가 값을 할당합니다.", 
           "B) 스테이지는 데이터베이스 테이블을 그룹화할 수 있습니다. 이를 위해 GARDEN_PLANTS 데이터베이스에 세 개의 스테이지를 생성했습니다. 하나는 VEGGIES로 이름을 지었습니다.", 
           "C) 스테이지는 Snowflake와 클라우드 폴더 간에 \"창\"을 제공할 수 있습니다."]                

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '255ef793eff139ae5abf8cbc1ce8a218'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "다음 중 Snowflake Cortex LLM Function이 아닌 것은?"
options = ["아래 보기 중 고르세요...",
           "A) TRANSLATE", 
           "B) SUMMARIZE", 
           "C) PARSE_DOCUMENT", 
           "D) COUNT_TOKENS", 
           "E) CONTENT", 
           "F) COMPLETE"]                

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'b71018c50689d88fc0f7aac8d3cbad47'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 축하합니다 :tada: :confetti_ball:

이 과정을 완료하셨습니다. - 정말 잘하셨습니다!